## <font color="green"> | - Questions théoriques :</font>

### Questions :

Relire le cours sur Elastic Search à l’aide de nos nouvelles connaissance et répondre aux questions:
1) Qu’est ce que le sharding, comment pourrait-on imaginer un sharding sur cet index?
2) Quels ingestion pipelines seraient pertinents pour notre sujet.


### Réponses :

    1) Le sharding est un concept clé dans Elasticsearch qui consiste à diviser un index en plusieurs fragments appelés shards.   
    Chaque shard est une unité autonome qui contient une partie des données de l'index.   
    Le sharding permet de distribuer les données sur plusieurs nœuds d'un cluster Elasticsearch, ce qui permet d'obtenir une meilleure répartition de la charge   
    et d'améliorer les performances de recherche et de récupération des données.


    Dans le cas de notre index, nous pouvons imaginer un sharding basé sur un critère tel que l'identifiant unique de chaque document.  
    Par exemple, nous pouvons choisir de sharder l'index en fonction d'une plage d'identifiants, de sorte que les documents ayant   
    des identifiants similaires soient regroupés dans le même shard. Cela permettrait de répartir la charge de manière équilibrée et de faciliter les opérations de recherche et d'agrégation.

    2) En ce qui concerne les pipelines d'ingestion, plusieurs options pourraient être pertinentes pour notre sujet. Voici quelques exemples :

        Le pipeline de normalisation : Ce pipeline pourrait inclure des étapes telles que la suppression des caractères spéciaux, la normalisation des mots en minuscules, la suppression des stopwords, etc. Cela permettrait de standardiser le texte avant l'indexation, ce qui faciliterait les opérations de recherche.

        Le pipeline de détection des émotions : Ce pipeline pourrait utiliser des techniques de traitement du langage naturel pour détecter les émotions présentes dans le texte, par exemple en utilisant des modèles de classification pré-entrainés ou des lexiques émotionnels. Cela permettrait d'extraire automatiquement les informations sur les émotions des textes lors de l'indexation.

        Le pipeline d'extraction des entités : Ce pipeline pourrait utiliser des techniques de reconnaissance d'entités nommées pour extraire des informations telles que les noms de personnes, les lieux, les organisations, etc. Cela permettrait d'enrichir les données avec des métadonnées supplémentaires, facilitant ainsi les requêtes spécifiques.

## <font color="green"> || - Alternatives :</font>

### Comment aurait-on pu intégrer la gestion des stopwords au niveau du Mapping? (proposez un code)?

Au niveau du Mapping, vous pouvez intégrer la gestion des stopwords en utilisant un "analyzer" personnalisé avec une liste de stopwords spécifiée. Voici un exemple de code pour intégrer la gestion des stopwords au niveau du Mapping dans Elasticsearch :

In [16]:
from elasticsearch import Elasticsearch

# Se connecter à Elasticsearch
es = Elasticsearch([{'host': 'localhost', 'port':9200, 'scheme':'http'}])

# Définir les stopwords personnalisés
custom_stopwords = ['ive','something','feel','feeling','feelings','like','im','know','get','would','time','little','even','one','life','people','think','bit','things','much','dont','make','going']

# Définir le mapping avec un analyseur personnalisé incluant les stopwords
mapping = {
    "properties": {
        "content": {
            "type": "text",
            "analyzer": "custom_analyzer"
        }
    },
    "settings": {
        "analysis": {
            "analyzer": {
                "custom_analyzer": {
                    "type": "standard",
                    "stopwords": custom_stopwords
                }
            }
        }
    }
}

# Créer l'index avec le mapping personnalisé
index_name = "notes2"
es.indices.create(index=notes2, body={"mappings": mapping})

# En appliquant ce mapping, le champ "content" sera analysé en utilisant l'analyseur "analyseur_personnalise", 
# qui supprimera les stopwords spécifiés lors de l'indexation et de la recherche.

NameError: name 'notes2' is not defined

### Comment aurait-on pu intégrer le modèle de ML comme ingest pipeline? (proposez un code)


In [10]:
import pickle
import pandas as pd
import texthero as hero
from texthero import stopwords, preprocessing
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from elasticsearch import Elasticsearch

# Load the trained model
with open('nlp_pipeline.pkl', 'rb') as file:
    nlp_pipeline = pickle.load(file)

# Define the prediction function
def predict_classification(text):
    return nlp_pipeline.predict([text])[0]

# Define the ingest processor
ingest_processor = {
    "set": {
        "field": "emotion",
        "value": {
            "script": {
                "source": """
                    def text = ctx.content;
                    return params.predict_function(text);
                """,
                "lang": "painless",
                "params": {
                    "predict_function": predict_classification
                }
            }
        }
    }
}

# Create the ingest pipeline
pipeline_name = "ml_classification_pipeline"
es.ingest.put_pipeline(id=pipeline_name, body={"processors": [ingest_processor]})

# Test the ingest pipeline
document = {
    "content": "Ceci est un exemple de texte à classifier"
}
result = es.index(index="notes2", body=document, pipeline=pipeline_name)


/tmp/ipykernel_22263/1911423459.py:38: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  es.ingest.put_pipeline(id=pipeline_name, body={"processors": [ingest_processor]})


SerializationError: Unable to serialize to JSON: {'processors': [{'set': {'field': 'emotion', 'value': {'script': {'source': '\n                    def text = ctx.content;\n                    return params.predict_function(text);\n                ', 'lang': 'painless', 'params': {'predict_function': <function predict_classification at 0x7f2371acf790>}}}}}]} (type: dict)